## DUSP1 Spot Detection Visulaization 

This Jupyter Notebook uses **Napari** to visualize pre-generated spot detection results in 3D. It utilizes **Dask** for efficient handling of large image datasets and integrates data from a mounted drive. The workflow includes loading masks and images, processing spot detection results, and rendering them interactively in Napari.


In [1]:
import napari
import matplotlib.pyplot as plt
import json
import pandas as pd
import h5py 
import numpy as np
import dask.array as da
import os
from tempfile import TemporaryDirectory
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis_DUSP1 import DUSP1AnalysisManager

/Users/ericron/Desktop/AngelFISH


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=26)
           2	RESUME(arg=0, lineno=26)
           4	LOAD_GLOBAL(arg=1, lineno=42)
          16	LOAD_FAST(arg=6, lineno=42)
          18	PRECALL(arg=1, lineno=42)
          22	CALL(arg=1, lineno=42)
          32	GET_ITER(arg=None, lineno=42)
>         34	FOR_ITER(arg=206, lineno=42)
          36	STORE_FAST(arg=7, lineno=42)
          38	LOAD_FAST(arg=0, lineno=43)
          40	LOAD_FAST(arg=3, lineno=43)
          42	LOAD_FAST(arg=5, lineno=43)
          44	BINARY_OP(arg=5, lineno=43)
          48	LOAD_FAST(arg=4, lineno=43)
          50	BINARY_OP(arg=0, lineno=43)
          54	COPY(arg=2, lineno=43)
          56	COPY(arg=2, lineno=43)
          58	BINARY_SUBSCR(arg=None, lineno=43)
          68	LOAD_CONST(arg=1, lineno=43)
          70	BINARY_OP(arg=13, lineno=43)
          74	SWAP(arg=3, lineno=43)
          76	SWAP(arg=2, lineno=43)
          78	STORE_SUBSCR(arg=None, lineno=43)
          82	LOAD_CONST(arg=

In [2]:
# Datafrmae location
df = pd.read_csv('/Volumes/share/Users/Eric/DUSP1_May2025/DUSP1_D_Final_FinalSpots_MG3_Abs9_May12_mg_abs.csv')
# H55 file location
h5_dir = '/Volumes/share/Users/Eric/DUSP1_H5/RepD'
h5_files = [
    os.path.join(h5_dir, f)
    for f in os.listdir(h5_dir)
    if f.endswith('.h5') and os.path.isfile(os.path.join(h5_dir, f))
]

In [3]:
am = DUSP1AnalysisManager(location=h5_files, mac=True) 
am.select_analysis('DUSP1_D_Final')

In [4]:
img, mask = am.get_images_and_masks()

In [5]:
for i, (r, m) in enumerate(zip(img, mask)):
    print(f"[{i}] raw_images: {type(r)}, shape={r.shape}, dtype={r.dtype}")
    print(f"[{i}] masks:      {type(m)}, shape={m.shape}, dtype={m.dtype}")

[0] raw_images: <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=float32
[0] masks:      <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=int8
[1] raw_images: <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=float32
[1] masks:      <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=int8
[2] raw_images: <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=float32
[2] masks:      <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=int8
[3] raw_images: <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=float32
[3] masks:      <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=int8
[4] raw_images: <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=float32
[4] masks:      <class 'dask.array.core.Array'>, shape=(30, 1, 3, 27, 936, 640), dtype=int8
[5] raw_images: <class 'dask.array.core.Array'>, shape=(30, 1, 3,

In [6]:
df.keys()

Index(['z_px', 'y_px', 'x_px', 'is_nuc', 'cluster_index', 'cell_label', 'snr',
       'signal', 'fov', 'FISH_Channel', 'condition', 'replica', 'time',
       'Dex_Conc', 'NAS_location', 'h5_idx', 'unique_spot_id',
       'unique_cell_id', 'cell_intensity_mean-0', 'cell_intensity_std-0',
       'nuc_intensity_mean-0', 'nuc_intensity_std-0', 'cyto_intensity_mean-0',
       'cyto_intensity_std-0', 'MG_SNR'],
      dtype='object')

In [ ]:
# 1) pick your file-index
h5_idx = 1
h5_path = h5_files[h5_idx]

# 2) extract the basename
fn = os.path.basename(h5_path)              
name_noext = fn                              

# 3) filter DataFrame by "endswith" that basename
df_h5 = df[df["NAS_location"].str.endswith(fn)].copy()

# sanity check
print("HDF5 file:  ", h5_path)
print("DF rows:    ", df_h5.shape[0], "points for", fn)

# 4) No need to re-index fovs: your 'fov' is already 0…29
#    Build your points array:
points = np.vstack([
    df_h5["fov"].values,
    df_h5["z_px"].values,
    df_h5["y_px"].values,
    df_h5["x_px"].values,
]).T

# 5) slice out just channel 0 from your raw dask stack
raw5d  = img[h5_idx]          # shape (30,1,3,27,936,640)
mrna4d = raw5d[:, 0, 0]       # now (30, 27, 936, 640)

# 6) launch Napari
viewer = napari.Viewer()
viewer.add_image(
    mrna4d,
    name=name_noext + "_mRNA",
    colormap="gray",
    scale=(5,1,1),
    channel_axis=None
)
viewer.add_points(
    points,
    name=name_noext + "_Spots",
    size=4,
    face_color="red",
    edge_color="red",
    scale=(5,1,1),
    ndim=4
)
napari.run()

DEBUG:in_n_out:Executing @injected ViewerToggleAction.__init__.<locals>.get_current(viewer: napari.viewer.Viewer) -> bool with args: (), kwargs: {}
DEBUG:in_n_out:  injecting viewer: <class 'napari.viewer.Viewer'> = Viewer(camera=Camera(center=(0.0, 0.0, 0.0), zoom=1.0, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=2, ndisplay=2, order=(0, 1), axis_labels=('0', '1'), rollable=(True, True), range=(RangeTuple(start=0.0, stop=2.0, step=1.0), RangeTuple(start=0.0, stop=2.0, step=1.0)), margin_left=(0.0, 0.0), margin_right=(0.0, 0.0), point=(np.float64(0.0), np.float64(0.0)), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[], help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_cal

HDF5 file:   /Volumes/share/Users/Eric/DUSP1_H5/RepD/DUSP1_Dex_120min_20220224.h5
DF rows:     66914 points for DUSP1_Dex_120min_20220224.h5


DEBUG:napari.components._layer_slicer:_LayerSlicer.submit: layers=[<Image layer 'DUSP1_Dex_120min_20220224.h5_mRNA' at 0x15dbfef50>], dims=ndim=4 ndisplay=2 order=(0, 1, 2, 3) axis_labels=('0', '1', '2', '3') rollable=(True, True, True, True) range=(RangeTuple(start=np.float64(0.0), stop=np.float64(29.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(130.0), step=np.float64(5.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(935.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(639.0), step=np.float64(1.0))) margin_left=(0.0, 0.0, 0.0, 0.0) margin_right=(0.0, 0.0, 0.0, 0.0) point=(np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)) last_used=0, force=False
DEBUG:napari.components._layer_slicer:Sync slicing for DUSP1_Dex_120min_20220224.h5_mRNA
DEBUG:napari.layers.base.base:Layer._slice_dims: DUSP1_Dex_120min_20220224.h5_mRNA, dims=ndim=4 ndisplay=2 order=(0, 1, 2, 3) axis_labels=('0', '1', '2', '3') rollable=(Tr

DEBUG:napari.components._layer_slicer:_LayerSlicer.submit: layers=[<Image layer 'DUSP1_Dex_120min_20220224.h5_mRNA' at 0x15dbfef50>, <Points layer 'DUSP1_Dex_120min_20220224.h5_Spots' at 0x3484cc650>], dims=ndim=4 ndisplay=2 order=(0, 1, 2, 3) axis_labels=('0', '1', '2', '3') rollable=(True, True, True, True) range=(RangeTuple(start=np.float64(0.0), stop=np.float64(29.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(130.0), step=np.float64(5.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(935.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(639.0), step=np.float64(1.0))) margin_left=(0.0, 0.0, 0.0, 0.0) margin_right=(0.0, 0.0, 0.0, 0.0) point=(np.float64(13.0), np.float64(65.0), np.float64(467.0), np.float64(319.0)) last_used=0, force=False
DEBUG:napari.components._layer_slicer:Sync slicing for DUSP1_Dex_120min_20220224.h5_mRNA
DEBUG:napari.components._layer_slicer:Sync slicing for DUSP1_Dex_120min_20220224.h5_Spots
DEBUG:n